## Homework

Write a generator for electron - positron pairs from the Z boson decay. Assume the Z boson is at rest, and its mass is ditributed according to the 
Breit-Wigner (BW) distribution with $m_{Z} = 92.1876$, $\Gamma = 2.4952$.

* generate events with the pair energy in range [50,200]
* assume the decay is isotropic (is this fully correct?)
* generate at least 500k events. How long it takes? What is the generation effciency?
* plot histogram of the pair invariant mass with linear and log Y scales. Overlay with probability distribution for invariant mass
* plot histogram of  $cos(\theta)$ of electrons moementum
* plot histogram of transverse electron momentum $p_{T}$. Overlay with probability distribution for transverse momentum.

**Hints:** 
* the probability distibution for the $p_{T}$ can be obtained from the chain rule:

\begin{equation}
\frac{d\sigma}{dp_{T}} =  \frac{d\sigma}{d\cos(\theta)} \frac{d\cos(\theta)}{dp_{T}}
\end{equation}

* use the nominal Z boson mass while calculating the $\frac{d\sigma}{dp_{T}}$.

Setup of our environment

In [2]:
#import ROOT

#Color printing
from termcolor import colored

#General data operations library
import math
import numpy as np

#HEP specific tools
import scipy.constants as scipy_constants

#Plotting libraries
import matplotlib.pyplot as plt

#Increase plots font size
params = {'legend.fontsize': 'xx-large',
          'figure.figsize': (10, 7),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(params)

In [3]:
G_F = scipy_constants.physical_constants["Fermi coupling constant"]
m_e = scipy_constants.physical_constants["electron mass energy equivalent in MeV"]
m_mu = scipy_constants.physical_constants["muon mass energy equivalent in MeV"]

print("Fermi constant: {} +- {} {}".format(G_F[0], G_F[2], G_F[1]))
print("muon mass: \t{} +- {} {}".format(m_mu[0], m_mu[2], m_mu[1]))
print("electron mass: \t{} +- {} {}".format(m_e[0], m_e[2], m_e[1]))

G_F = G_F[0]
m_e = m_e[0]
m_mu = m_mu[0]

Fermi constant: 1.1663787e-05 +- 6e-12 GeV^-2
muon mass: 	105.6583755 +- 2.3e-06 MeV
electron mass: 	0.51099895 +- 1.5e-10 MeV


## Our homework functions

In [4]:
minS = 50
maxS = 200

m_Z = 92.1876
gamma_Z = 2.4952

def BreitWigner(s, m, gamma):
    gamma_small = np.sqrt(m ** 2 * (m ** 2 + gamma ** 2))
    k = (2 * np.sqrt(2) * m * gamma * gamma_small) / (np.pi * np.sqrt(m ** 2 + gamma_small))
    
    distr = k / ((s ** 2 - m ** 2) ** 2 + m ** 2 * gamma ** 2)
    return distr

def generateCandidateEnergy(nPoints):
    gamma = gamma_Z
    s = (maxS-minS)*np.random.random(nPoints) + minS
    print(s)
    x = np.random.random_sample(nPoints)
    bw_values = BreitWigner(s, m_Z, gamma)  # it is equivalent to p
    bw_values = bw_values / np.max(bw_values)
    accepted = x < bw_values
    return s[accepted]/2.0

def generateCandidateMomenta(energy):
    howmany = len(energy)
    
    c = np.random.random_sample([howmany, 3]) 
    csum = np.sum(c, axis=1)  # we need to generate shoots so that their sum equals energy
    cdiv = np.asanyarray(1 / csum)  # vector we multiply our matrix
    cdivT = np.vstack(cdiv)  # transposed vector

    result = np.multiply(c, cdivt)  # matrix multiplied  by sum
    energyT = np.vstack(energy)  # transposed vector of energies
    
    p = np.multiply(result, energyT)
    print(p)
       
    p4_electron = np.column_stack([energy, px,py,pz])
    p4_positron = np.column_stack([energy, -px,-py,-pz])
    return  p4_electron, p4_positron

def generateEvents(nPoints):
    energy = generateCandidateEnergy(nPoints)
    data = generateCandidateMomenta(energy)
    return data

def invMass(p4):
    metric = np.array([1,-1,-1,-1])
    p4_square = p4*(metric*p4)
    m = np.sqrt(np.sum(p4_square, axis=1))
    return m

## I have done here multiplication of two arrays; each row is multiplied by different value; thanks to this now we can generate N rows (for each energies) consisting of 3 momentum values


In [5]:
energy = np.array([1,2,3,4,5])
howmany = 5

c = np.random.random_sample([howmany, 3]) 
csum = np.sum(c, axis=1)  # we need to generate shoots so that their sum equals energy
cdiv = np.asanyarray(1 / csum)  # vector we multiply our matrix
cdivT = np.vstack(cdiv)  # transposed vector

result = np.multiply(c, cdivT)  # matrix multiplied  by sum
energyT = np.vstack(energy)  # transposed vector of energies

p = np.multiply(result, energyT)

a = np.array([[1, 1, 1, 1], [2, 2, 2, 2]])
b = np.array([[2, 2, 2 ,2], [2, 2, 2 ,2]])



d = np.column_stack((a))

print(c)
# hellos a

[[0.7858976  0.4014081  0.26992839]
 [0.07467526 0.10604936 0.72553995]
 [0.62994019 0.56251426 0.68507149]
 [0.05640179 0.55865698 0.15418465]
 [0.65544372 0.53027935 0.9893821 ]]


In [6]:
c = np.random.random_sample([5, 3]) 

csum = np.sum(c, axis=1)
print("\nMatrix c : \n", c)
print(csum)

cdiv = np.asanyarray(1 / csum)
cdivt = np.vstack(cdiv)
print(cdivt)

r = np.multiply(c, cdivt)



Matrix c : 
 [[0.76516363 0.76546357 0.8258116 ]
 [0.36670747 0.79994341 0.55293624]
 [0.34495911 0.50473095 0.60484639]
 [0.30899205 0.48361058 0.4702391 ]
 [0.67031036 0.7537014  0.24333733]]
[2.3564388  1.71958711 1.45453646 1.26284174 1.66734909]
[[0.42436918]
 [0.58153495]
 [0.68750425]
 [0.79186486]
 [0.59975443]]


## Here are energies generated

In [7]:
nPoints = 50000

e_min = 50
e_max = 200

energies = generateCandidateEnergy(nPoints)
print("Number of accepted events:\t",energies.shape[0])
print("Accepted events fraction:\t",energies.shape[0]/nPoints)

energies 


[124.67720003  98.55320551 127.34431158 ... 187.45563644  81.2122681
 189.03600296]
Number of accepted events:	 1324
Accepted events fraction:	 0.02648


array([43.05915604, 45.78658891, 45.54566874, ..., 43.64373894,
       47.11139561, 45.34167   ])

## Here is working generating momenta


In [8]:
b = generateCandidateMomenta(energies)
b

ValueError: operands could not be broadcast together with shapes (1324,3) (5,1) 

## First test plots

In [ ]:
%%time

nPoints = 1000000
p4_electron, p4_positron = generateEvents(nPoints)
m = invMass(p4_electron + p4_positron)
#print("Generated {} events for {} tries. \nEfficiency: {:3.2f}".format(m.shape[0], nPoints, float(m.shape[0])/nPoints))

#fig, axes = plt.subplots(2,2, figsize=(12, 12))
#axes[0,0].plot(m, BreitWigner(m, m_Z, gamma_Z), "o",label="Breit-wigner");
#axes[0,0].hist(..., label=r"$e^{+}e^{-}$ MC data");
#axes[0,0].set_xlabel(r"invariant mass [GeV/$c^{2}$]")
#axes[0,0].set_ylabel("probability density")
#axes[0,0].legend(loc="upper right");